<a href="https://colab.research.google.com/github/dlskawns/cp1/blob/main/5_DCN__RecSys_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. 특성을 활용한 딥러닝 기반 추천 시스템 모델


## DCN - Deep Cross NET 이용 

CTR(클릭률)에 활용되는 모델로 다양한 특성들의 조합을 고려할 수 있는 모델

### * 선정 이유:  
유저특성, 상품특성을 인풋데이터로 넣어 Embedding, Cross, DNN Layer로 이어지는 Stacked 구조의 모델을 거쳐  
각 feature의 조합에 따라 라벨을 예측하는 지도학습을 진행하기 위함.


### * 가설:  
Deep Cross Net의 구조를 이용해 모델을 설계하면, 인풋 데이터의 모든 Feature들에 대한 특성을 고려해 학습 및 예측이 가능할 것이다.


### * 방법:  
* 인풋 데이터
  * Score를 바탕으로 학습 및 예측(추천)하기 위한 label 생성
    * 전체 평균 Score(4.18)에 따라 labelling -> 4점 이하 = 0, 5점 = 1
  * UserId, ItemId뿐 아니라 유저특성, 상품특성을 Embedding Layer의 인풋데이터로 넣는다.
  * 추천 시스템을 고려해서 학습 특성 선택
* 모델 아키텍쳐
  * 인풋 데이터의 각 특성 별로 Embedding Layer를 작성해넣고, 이를 concat하여 합쳐준다.
  * Cross Layer를 작성한 뒤, Embedding Layer의 아웃풋을 인풋으로 통과한다.
  * Deep Layer를 작성한 뒤, Cross Layer의 아웃풋을 인풋으로 넣어 최종 분류기(binary-sigmoid)를 통과시킨다.
  * metric = BinaryAccuracy
* 추천 모듈(recommendation)
  * UserId 입력시 users(유저 룩업테이블)에서 DCN 모델 predict 진행을 위한 feature 정보를 가져온다. (UserId, 리뷰 수, 반려동물 유무)
  * items(상품 룩업테이블)의 상품에 대한 label을 예측한다.
  * 예측 완료 후 높은 순위의 확률을 가진 10개 상품을 노출시킨다.

  <br>

  ---

### 필요 모듈 가져오기 

In [ ]:
!pip install -q tensorflow-recommenders

     |████████████████████████████████| 85 kB 2.9 MB/s 


In [ ]:
import os
import sys
import gc
import glob
import joblib
from google.colab import drive
from tqdm import tqdm

import numpy as np
import pandas as pd

import keras
import tensorflow as tf
import tensorflow_recommenders as tfrs

from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, Callback
from tensorflow.keras.utils import plot_model

from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow_datasets as tfds
import pandas as pd

### 데이터 불러오기

* 구글 드라이브의 폴더 및 파일을 접근하기 위해 마운트 합니다.

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle

with open('/content/drive/MyDrive/cp1_data.pkl', 'rb') as f:
  df = pickle.load(f)

### 학습 데이터 설정하기

#### 학습 진행을 위한 샘플 별 label 값 설정

* 유저의 평점(Score)의 평균값 = 4.16  
* 전체 평점의 평균값 = 4.18  
ㄴ> 전체적으로 5점 분포가 가장 높기 때문에 4점 이하를 label 0, 5점을 label 1로 설정하여 진행

In [ ]:
df.groupby('ProductId')['Score'].mean().mean()  # 상품별 평균 평점의 평균

4.1661890814501445

In [ ]:
df['Score'].mean()  # 상품 리뷰 샘플 전체의 평균

In [ ]:
# New labeling
label = []
for i in df['Score']:
  if i >4:
    label.append(1)
  else:
    label.append(0)
df['label'] = label

In [ ]:
# pet label을 str으로 바꿔줌
for i in range(len(df['pet'])):
  if df['pet'][i] == 0:
    df['pet'][i] = 'no'
  elif df['pet'][i] == 1:
    df['pet'] = 'dog'
  elif df['pet'][i] == 2:
    df['pet'] = 'cat'

#### Model Input 값 및 User, Item feature 테이블 설정

* 필요 특성을 미리 데이터셋에 추가해준다.
* 추후 추천 진행을 위해 유저 및 상품의 고유한 특성 목록을 생성한다.

  * 유저 Features: 
    * UserId(고객ID) 
    * r_counts(유저의 리뷰 총 개수)
    * pet(반려동물 유무)
  * 상품 Features: 
    * ProductId(상품ID)
    * r_counts_pro(상품의 리뷰 개수)
    * review_len_average(상품 리뷰의 평균 길이)




In [ ]:
# 리뷰 길이 feature 생성
df['review_len'] = df['Text'].apply(lambda x : len(x))

In [ ]:
# 추천 및 학습을 위한 상품feature 테이블 생성
item_c = pd.DataFrame(df['ProductId'].value_counts()).reset_index().rename(columns = {'ProductId': 'r_counts_pro','index':'ProductId'})
item_m = pd.DataFrame(df.groupby('ProductId')['review_len'].mean()).reset_index()
items = item_c.merge(item_m, 'left',on = 'ProductId').rename(columns={'review_len':'review_len_average'})
df = df.merge(items, 'left', on = 'ProductId')

In [ ]:
# 추천 및 학습을 위한 유저feature 테이블 생성
users = df[['UserId','r_counts','pet']]
users = users.drop_duplicates()

#### 모델 컴퓨팅 클래스 작성

In [ ]:
class model(tfrs.Model):
    """
    model class로 tfrs.Model을 상속해 call함수를 통해 모델 아키텍쳐 생성 

    """


    def __init__(self, deep_layer_sizes, learning_rate, str_features, int_features, vocabularies, projection_dim = None, metric = 'binary'):
        super().__init__()
    
        self.embedding_dimension = 64
    
        self._all_features = str_features + int_features
        self._embeddings = {}
    
        # Compute embeddings for string features.
        for feature_name in str_features:
            vocabulary = vocabularies[feature_name]
            self._embeddings[feature_name] = tf.keras.Sequential(
                [tf.keras.layers.experimental.preprocessing.StringLookup(
                    vocabulary=vocabulary, mask_token=None),
                    tf.keras.layers.Embedding(len(vocabulary) + 1,
                    self.embedding_dimension)])
          
    
        # Compute embeddings for int features.
        for feature_name in int_features:
            vocabulary = vocabularies[feature_name]
            self._embeddings[feature_name] = tf.keras.Sequential(
                [tf.keras.layers.experimental.preprocessing.IntegerLookup(
                    vocabulary=vocabulary, mask_value=None),
                    tf.keras.layers.Embedding(len(vocabulary) + 1,self.embedding_dimension)])
        
        # Cross layer

        self._cross_layer = tfrs.layers.dcn.Cross(
            projection_dim = projection_dim,
            kernel_initializer = "glorot_uniform") 
            
        # Deep layer
        self._deep_layers = [tf.keras.layers.Dense(layer_size, activation="relu")
            for layer_size in deep_layer_sizes]
        
        # Output layer
        self._logit_layer = tf.keras.layers.Dense(1,activation = 'sigmoid')
        # Metric
        if metric == 'binary':
            self.task = tfrs.tasks.Ranking(
            loss = tf.keras.losses.BinaryCrossentropy(),
            metrics=[
                    tf.keras.metrics.BinaryAccuracy(name='binary_accuracy', dtype = None, threshold = 0.5)])
    
    def call(self, features):
        # Concatenate embeddings
        embeddings = []
        for feature_name in self._all_features:
            embedding_fn = self._embeddings[feature_name]
            embeddings.append(embedding_fn(features[feature_name]))
    
        x = tf.concat(embeddings, axis=1)
    
        # Build Cross Network
        cross_layer = self._cross_layer:
        x = cross_layer(x)
        
        # Build Deep Network
        for deep_layer in self._deep_layers:
            x = deep_layer(x)

        return self._logit_layer(x)
    
    def compute_loss(self, features, training=False, metric = 'binary'):
        if metric == 'binary':
            labels = features.pop("label")
        scores = self(features)
    
        return self.task(labels=labels,predictions=scores)

#### 모델 인풋값 전처리 함수 작성

In [ ]:
import os
import sys
import gc
import glob
import joblib
from tqdm import tqdm

import ast
import numpy as np
import pandas as pd

import keras
import tensorflow as tf


def DCN(df, str_features, int_features, df_type = 'train'):
"""
인풋값을 만들기 위한 함수.
train용과 test용으로 나뉘어져있음.
"""

    feature_names = str_features + int_features

    # feature type 변경
    def setType(df):
        for f in str_features:
            if df[f].dtype == float:
                df[f] = df[f].astype(int)

        for f in int_features:
            df[f] = df[f].astype(int)
            
        return df
  
    # 데이터 dict로 변환
    def generateDict(df):
        # str features는 encoding
        train_str_dict = {str_feature: [str(val).encode() for val in df[str_feature].values]for str_feature in str_features}
        # int features는 int
        train_int_dict = {int_feature: df[int_feature].valuesfor int_feature in int_features}

        # # label columns이 있다면~
        try:
            train_label_dict = {'label' : df['label'].values}
            train_str_dict.update(train_label_dict)
        except:
            pass

        train_str_dict.update(train_int_dict)
        return train_str_dict


    df_copy = setType(df)
    input_dict = generateDict(df_copy)

    # tensor
    tensor = tf.data.Dataset.from_tensor_slices(input_dict)
    cached = tensor.shuffle(100_000).batch(8192).cache()
    # unique data 저장
    # train data 일 때, 
    if df_type == 'train':
        vocabularies = {}
    
        for feature_name in tqdm(feature_names):
            vocab = tensor.batch(1_000_000).map(lambda x: x[feature_name])
            vocabularies[feature_name] = np.unique(np.concatenate(list(vocab)))
    
        return cached, vocabularies
      
      # test data 일 때, 
    else:
        return cached

In [ ]:
# Input feature 설정
str_features = ['ProductId','UserId','pet'] # 부득이 keyword를 넣지 못함 
int_features = ['r_counts','r_counts_pro','review_len_average']
label_feature=["label"]
feature_names = str_features + int_features + label_feature

In [ ]:
df2 = df.copy()
df2 = df.sample(frac=1) # 샘플 순서를 섞어준다.

In [ ]:
from sklearn.model_selection import train_test_split

# 학습, 검증 셋 설정
train, val = train_test_split(df2, test_size = 0.2, random_state = 2)
val, test = train_test_split(val, test_size = 0.2, random_state = 2)

In [ ]:
# 라벨 1, 라벨 0 의 분포 확인
train.label.value_counts()

1    290652
0    164111
Name: label, dtype: int64

In [ ]:
# 학습 데이터셋 label 분포에 맞춰 under sampling 진행 
train_0 = train[train['label']==0]
train_1= train[train['label']==1]
train_1 =train_1.sample(frac=1)
train_11 =train_1.iloc[:len(train_0)]
train = pd.concat([train_0,train_11], axis=0)

In [ ]:
train.label.value_counts()

0    164157
1    164157
Name: label, dtype: int64

In [ ]:
# 전처리 함수를 통해 embedding용 vocab 및 학습 셋, 검증 셋 생성
cached_train, vocabularies = DCN(train, str_features, int_features, df_type = 'train')
cached_val = DCN(val, str_features, int_features, df_type = 'val')
cached_test = DCN(test, str_features, int_features, df_type = 'test')

### 모델 생성 및 컴파일

In [ ]:
learning_rate = 0.0001
model = model(deep_layer_sizes = [512, 256, 128, 64, 32, 16,8],
            learning_rate = learning_rate,
            str_features = str_features,
            int_features = int_features,
            vocabularies = vocabularies,
            projection_dim = None,
            metric = 'binary'
            )

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate))

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_16 (Sequential)  (None, 64)                192       
                                                                 
 sequential_12 (Sequential)  (None, 64)                3733824   
                                                                 
 sequential_13 (Sequential)  (None, 64)                11005952  
                                                                 
 sequential_14 (Sequential)  (None, 64)                128       
                                                                 
 sequential_15 (Sequential)  (None, 64)                9280      
                                                                 
 sequential_18 (Sequential)  (None, 64)                18176     
                                                                 
 sequential_17 (Sequential)  (None, 64)                2252

In [ ]:
import keras
callbacks_list = [
                  keras.callbacks.EarlyStopping(
                      monitor = 'val_loss',
                      patience = 5
                      )]

#### 모델 학습

In [ ]:
history0 = model.fit(cached_train,
                     epochs = 40,
                     callbacks = callbacks_list,
                     verbose = True, validation_data = cached_val)

Epoch 1/40
41/41 [==============================] - 1s 28ms/step - binary_accuracy: 0.4934 - loss: 0.6989 - regularization_loss: 0.0000e+00 - total_loss: 0.6989 - val_binary_accuracy: 0.3607 - val_loss: 0.6956 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.6956
Epoch 2/40
41/41 [==============================] - 1s 23ms/step - binary_accuracy: 0.4931 - loss: 0.6963 - regularization_loss: 0.0000e+00 - total_loss: 0.6963 - val_binary_accuracy: 0.3607 - val_loss: 0.6946 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.6946
Epoch 3/40
41/41 [==============================] - 1s 23ms/step - binary_accuracy: 0.4934 - loss: 0.6946 - regularization_loss: 0.0000e+00 - total_loss: 0.6946 - val_binary_accuracy: 0.3607 - val_loss: 0.6940 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.6940
Epoch 4/40
41/41 [==============================] - 1s 23ms/step - binary_accuracy: 0.4934 - loss: 0.6942 - regularization_loss: 0.0000e+00 - total_loss: 0.6942 - val_binary_accurac

#### 평가 및 추천 진행 함수 작성

In [ ]:
def getResult(model, cached_test, metric = 'binary'):
    
    cached_test_numpy = tfds.as_numpy(cached_test)
    y_true = [item['label'] for item in cached_test_numpy]
    y_true = np.concatenate(y_true)

    y_pred = model.predict(cached_test).flatten()

    if metric == 'binary':
        y_pred_class = [1 if pred > 0.5 else 0 for pred in y_pred]

    print(f"confusion: {confusion_matrix(y_true, y_pred_class)}")
    print(classification_report(y_true, y_pred_class))


def recommendation(userID, model,users, items, str_features, int_features):
    items_copy = items.copy()
    items_copy['UserId'] = userID
    items_copy = items_copy.merge(users, 'left', on= 'UserId')
    
    cached = DCN(items_copy, str_features, int_features, df_type = 'test')
    p = model.predict(cached)
    items_copy['prob'] = p
    items_copy = items_copy.sort_values(by = 'prob', ascending = False).reset_index()
    rec_lst = items_copy.head(10)['ProductId'].values


    return rec_lst

In [ ]:
model.evaluate(cached_val)

12/12 [==============================] - 0s 7ms/step - binary_accuracy: 0.7505 - loss: 0.7936 - regularization_loss: 0.0000e+00 - total_loss: 0.7936


[0.7505112290382385, 0.7801525592803955, 0, 0.7801525592803955]

In [ ]:
getResult(model, cached_test, metric = 'binary')

confusion:
 [[ 4991  3295]
 [ 2113 12340]]
              precision    recall  f1-score   support

           0       0.70      0.60      0.65      8286
           1       0.79      0.85      0.82     14453

    accuracy                           0.76     22739
   macro avg       0.75      0.73      0.73     22739
weighted avg       0.76      0.76      0.76     22739



### 상품 추천 시연




In [ ]:
# 임의의 유저 선정
import random
a = random.choice(users['UserId'].values)
print(a)

A25Z86FX7LN6G


In [ ]:
model.recommendation(a)

array(['B000ISZ310', 'B000FA15GI', 'B005LQ4XK6', 'B002M5GK14',
       'B004346KYY', 'B001UHRX1G', 'B000F4J74G', 'B003U4M4ZC',
       'B001D6B1SU', 'B00238ZZDO'], dtype=object)

In [ ]:
df[df['UserId']== 'A25Z86FX7LN6G']

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,date,dayofweek,r_counts,keyword,labels,pet,Help,label,review_len,r_counts_pro,review_len_average
200191,200192,B003Z6ZHJK,A3CZEEXWJL032W,=shocked=,1,5,1,1319500800,tastes watered down,=Has anyone noticed that the grape juice in th...,2011-10-25,Tuesday,1,"['plastic', 'noticed', 'grape', 'juice', 'cont...",NaN,dog,0.2,0,133,1,133.0


In [ ]:
# 해당 유저가 구매했던 제품의 순위
items_copy[items_copy['ProductId']== 'B003Z6ZHJK']

,index,ProductId,r_counts_pro,review_len_average,UserId,r_counts,pet,prob
35089,45417,B003Z6ZHJK,1,133,A3CZEEXWJL032W,1,dog,0.499974


In [ ]:
items_copy[items_copy['ProductId']== 'B000GUOA8W']

,ProductId,r_counts_pro,review_len_average,UserId,r_counts,pet,prob
6640,B000GUOA8W,15,385,A1LPBZLHP4YP66,1,dog,0.499975


### * DCN 모델 학습 및 평가 결과:
  * val_accuracy 75% 정도의 성능을 보인다.
  * 추천을 담당하는 라벨 1의 예측이 좀 더 잘 되는 편이며, recall이 더 높게나오므로 잘못된 상품 추천을 줄일 수 있을 것이라 판단.
  * 라벨 0의 예측은 f1 65% 정도로 recall이 좀 낮게 나오는 편.




### * 상품 추천 결과:
  * 고객이 구매하지 않았던 내용에 대해서도 결과값이 확인이 됨.
  * pet feature가 일치한 경우가 많음 ex: dog일 경우 기존 데이터 셋 상에서도 dog인 경우





### * 문제 및 개선점:
  * 상품명이 없기 때문에 상품 추천 결과 자체의 품질을 파악하기 어려움.
  * 학습 단계에서부터 최고 성능이 76%정도로 높지 않은 성능을 유지함.
    * 원인 1: DCN 모델을 사용하기에 feature 수가 너무 적음.  
      ㄴ> 좀 더 많은 유저 or 상품 특성을 추가해 학습해야 개선 가능
    * 원인 2: label이 정보에 비해 매우 불균등한 상황 (1~4점까지 label 0)  
      ㄴ> 다음 문제점에 대한 개선과 같이 
  * label(0,1)의 설정이 전체 score의 평균(4.18)을 기준으로 하여 4점이어도 추천이 되지 않는 상황이 발생  
    * 원인 : 기존 데이터가 리뷰에 초점이 맞춰져 있어 고객들의 상품에 대한 score가 매우 불균등함.
      * 균등하게 smote하거나 under sampling 하여1\~3에 label 0로 부여, 4~5에 label 1을 부여한다.

